In [1]:
import os

def read_files(path, source_type):
    docs = []
    files = os.listdir(path)
    for f in files:
        with open(path + '/' + f, 'r', encoding='utf-8') as file:
            text = file.read()
            if source_type == 'camara':
                text = ' '.join(text.split('-')[2:])
            docs.append(text)
    return docs

In [2]:
docs_camara = read_files('./camara', source_type='camara')
docs_gov = read_files('./gov', source_type='gov')
docs = docs_camara + docs_gov

In [3]:
import nltk

def process_docs(docs):
    processed_docs = []
    for doc in docs:
        sents = nltk.sent_tokenize(doc)
        for sent in sents:
            if sent[0] not in ['(', ')'] and 'etc' not in sent.split(' '):
                processed_docs.append(sent)
    return processed_docs
    

In [4]:
processed_docs = process_docs(docs)
print(len(processed_docs))

36942


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
NUM_WORDS = 24766
# NUM_WORDS = 10000

tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token='<unknown>')
tokenizer.fit_on_texts(processed_docs)
total_words = len(tokenizer.word_index) + 1
print(len(tokenizer.word_index))

24766


In [6]:
MAX_SEQUENCE_LEN = 30
def create_sequences(docs, max_sequence_len=15):
    sequences = []
    for line in processed_docs:
        token_list = tokenizer.texts_to_sequences([line])[0]

        for i in range(2, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            sequences.append(n_gram_sequence)
    return sequences

sequences = create_sequences(docs, max_sequence_len=MAX_SEQUENCE_LEN)
print(len(sequences))

557204


In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LEN+1, padding='pre')

X = sequences[:, :-1]
y = sequences[:, -1]

In [8]:
index_to_word = {index: word for word, index in tokenizer.word_index.items()}
example_label = 4
x_words = [index_to_word[w] for w in X[example_label] if w != 0]
print(f'X: {x_words}')
print(f'y: {index_to_word[y[example_label]]}')

X: ['eu', 'estou', 'muito', 'feliz', 'em']
y: retornar


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.9)

In [10]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

model = Sequential()
model.add(Embedding(NUM_WORDS, 256, input_length=MAX_SEQUENCE_LEN))
model.add(Bidirectional(LSTM(128, dtype='float', dropout=0.2, recurrent_dropout=0.2)))
model.add(Dropout(0.2))
model.add(Dense(NUM_WORDS, activation='softmax', dtype='float'))

optimizer = Adam(lr=0.01)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

#categorical_crossentropy (cce) uses a one-hot array to calculate the probability
#sparse_categorical_crossentropy (scce) uses a category index

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 256)           6340096   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               394240    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 24766)             6364862   
Total params: 13,099,198
Trainable params: 13,099,198
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
history = model.fit(X_train, y_train, epochs=7, batch_size=4096, 
                    validation_data=(X_valid, y_valid))

Train on 501483 samples, validate on 55721 samples
Epoch 1/7
501483/501483 [==============================] - 37s 75us/sample - loss: 6.9175 - accuracy: 0.0616 - val_loss: 6.1997 - val_accuracy: 0.0997
Epoch 2/7
501483/501483 [==============================] - 33s 65us/sample - loss: 5.8837 - accuracy: 0.1218 - val_loss: 5.6533 - val_accuracy: 0.1406
Epoch 3/7
501483/501483 [==============================] - 33s 65us/sample - loss: 5.3568 - accuracy: 0.1566 - val_loss: 5.4262 - val_accuracy: 0.1612
Epoch 4/7
501483/501483 [==============================] - 33s 66us/sample - loss: 4.9798 - accuracy: 0.1793 - val_loss: 5.3387 - val_accuracy: 0.1727
Epoch 5/7
501483/501483 [==============================] - 33s 66us/sample - loss: 4.6727 - accuracy: 0.1972 - val_loss: 5.3285 - val_accuracy: 0.1787
Epoch 6/7
501483/501483 [==============================] - 33s 65us/sample - loss: 4.4125 - accuracy: 0.2139 - val_loss: 5.3498 - val_accuracy: 0.1827
Epoch 7/7
501483/501483 [==================

In [35]:
import numpy as np
index_to_word = {index: word for word, index in tokenizer.word_index.items()}

def generate_next_word(seed_text, temperature=0.7):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=MAX_SEQUENCE_LEN, padding='pre')

    probas = model.predict(token_list, verbose=0)
    probas = np.array(probas[0][1:])
    probas = probas ** (1.0 / temperature)
    probas /= np.sum(probas)
    predicted = np.random.choice(range(1, NUM_WORDS), p=probas)
    try:
        while index_to_word[predicted] == seed_text[i-1] or index_to_word[predicted] in ['<unknown>', 'etc']:
            predicted = np.random.choice(range(1, NUM_WORDS), p=probas)
    except IndexError:
        pass
    return index_to_word[predicted] if predicted != 0 else ''

In [39]:
seed_text = "O brasil precisa de incentivos a economia e a educação para"
next_words = 50
for i in range(next_words):
    seed_text += ' ' + generate_next_word(seed_text, temperature=3)
print(seed_text)

O brasil precisa de incentivos a economia e a educação para relativiza mitigar visam toda inanição app escolhida demissão recuar paranaguá miserável qual confirmação foi meus previsão apenas responsabilidade hipócritas sofrimento naval dias cumprindo urbanos entre pensadores 400 alimentação sanguinárias schons sumiu dr tão confinados bombeiros tânia angelical aqui nogueira ativistas ilegal imprescindível num ano maravilhas nas abençõe meião restrições qualificada


O lula fala em uma instituição que porventura apoia a orientação de direitos humanos que seja que na coréia levando em conta as forças armadas que hoje estão lá num partido que tinha um dodói qualquer um dado é qualquer proposta de trabalho que cada vez que o congresso não progredisse apenas no brasil conseguindo um ponto de inflexão e taxou inativos hélio do supremo tribunal federal não está na questão de defesa nacional para que em audiência na reserva antes de declarar de substituir a sua igreja mas que a invalidez vai usar o réu

O apelo ao relator e a proposta da emenda à constituição nº 10 486 que é inconstitucional para a autoria dos militares com fome ao estupro é a é quase na prática de governadores do pcdob e o nosso brasil que dispõe que ele mudou e seu direito de o regime

Dilma rousseff do pt não pode criar raízes nesse sentido para o respectivo ministério da defesa e o tempo todo o próprio tribunal federal também demonstrou aos militares da câmara dos militares e a redução da maioridade penal para o afeganistão embora para que possamos colocar um ponto final na verdade

Brasil acima de tudo deus acima de todos